In [12]:
import json
import pandas as pd
import re

with open('flats.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Helper functions for cleaning/extracting data
def clean_area(area_str):
    """Remove non-breaking spaces and ' м²', replace comma with dot,
       then convert to a number. If the result is an integer, cast to int."""
    if area_str:
        area_str = area_str.replace('\xa0', ' ').replace(" м²", "").strip()
        area_str = area_str.replace(",", ".")
        value = float(area_str)
        if value.is_integer():
            return int(value)
        return value
    return None

def parse_floor(floor_str):
    """Split floor string like '1 из 5' into (flat_floor, total_floors)."""
    parts = re.split(r'из', floor_str)
    if len(parts) == 2:
        flat_floor = int(parts[0].strip())
        total_floors = int(parts[1].strip())
        return flat_floor, total_floors
    return None, None

def extract_metro_time(metro_dict):
    """Return the minimum travel time from the metro dictionary, ignoring None values."""
    if metro_dict:
        valid_times = [t for t in metro_dict.values() if t is not None]
        if valid_times:
            return min(valid_times)
    return None

def extract_rooms(title):
    """Extract the number of rooms from the title.
       Return 'студия' if mentioned, otherwise the digit preceding '-комн.'."""
    if "студия" in title.lower():
        return "студия"
    match = re.search(r'(\d+)-\s*комн', title.lower())
    if match:
        return int(match.group(1))
    return None

def clean_ceiling_height(height_str):
    """Clean and convert ceiling height string like '3\xa0м' to a float."""
    if height_str:
        height_str = height_str.replace('\xa0', ' ').replace(" м", "").strip().replace(",", ".")
        return float(height_str)
    return None

def extract_int(value_str):
    """Extract the first integer found in the string."""
    if value_str:
        digits = re.findall(r'\d+', value_str)
        if digits:
            return int(digits[0])
    return None

# Process each record and build a list of rows for the DataFrame
rows = []
for d in data:
    row = {}
    # Top-level fields
    row["okrug"] = d.get("okrug")
    row["raion"] = d.get("raion")
    row["price"] = d.get("price")
    row["living_complex"] = d.get("living_complex")
    row["title"] = d.get("title")
    row["rooms"] = extract_rooms(d.get("title", ""))
    
    # Process metro: take the closest (minimum) time
    row["metro_time"] = extract_metro_time(d.get("metro", {}))
    
    # Process 'info' fields
    info = d.get("info", {})
    row["info_total_area"] = clean_area(info.get("Общая площадь"))
    row["info_living_area"] = clean_area(info.get("Жилая площадь"))
    row["info_kitchen_area"] = clean_area(info.get("Площадь кухни"))
    flat_floor, total_floors = parse_floor(info.get("Этаж", ""))
    row["info_floor"] = flat_floor
    row["info_total_floors"] = total_floors
    row["info_year_built"] = int(info.get("Год постройки").strip()) if info.get("Год постройки") else None
    
    # Process 'flat_details' fields:
    fd = d.get("flat_details", {})
    # Define keys to extract with mapping: JSON key -> (column name, processing function)
    fd_keys_to_extract = {
        "Общая площадь": ("fd_total_area", clean_area),
        "Жилая площадь": ("fd_living_area", clean_area),
        "Площадь кухни": ("fd_kitchen_area", clean_area),
        "Высота потолков": ("ceiling_height", clean_ceiling_height),
        "Санузел": ("bathroom", lambda x: x),
        "Вид из окон": ("view", lambda x: x),
        "Ремонт": ("repair", lambda x: x),
        "Год постройки": ("fd_year_built", lambda x: int(x.strip()) if x else None),
        "Строительная серия": ("building_series", lambda x: x),
        "Мусоропровод": ("waste_chute", lambda x: x),
        "Тип дома": ("house_type", lambda x: x),
        "Тип перекрытий": ("ceiling_type", lambda x: x),
        "Парковка": ("parking", lambda x: x),
        "Подъезды": ("entrances", extract_int),
        "Отопление": ("heating", lambda x: x),
        "Аварийность": ("emergency", lambda x: x),
        "Газоснабжение": ("gas_supply", lambda x: x),
        "Тип жилья": ("housing_type", lambda x: x),
        "Балкон/лоджия": ("balcony", extract_int),
        "Количество лифтов": ("num_lifts", extract_int)
    }
    

    # Process 'rent_details' fields (new fields)
    rent = d.get("rent_details", {})
    row["rent_price_per_month"] = rent.get("price_per_month")
    row["rent_jkh"] = rent.get("Оплата ЖКХ")
    row["rent_deposit"] = rent.get("Залог")
    row["rent_commissions"] = rent.get("Комиссии")
    row["rent_prepayment"] = rent.get("Предоплата")
    row["rent_term"] = rent.get("Срок аренды")
    row["rent_living_conditions"] = rent.get("Условия проживания")
    
    rows.append(row)

# Create the DataFrame and save as CSV
df = pd.DataFrame(rows)
df.to_csv('final_data.csv', index=False, encoding='utf-8')


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1405 entries, 0 to 1404
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   okrug                   1405 non-null   object 
 1   raion                   1405 non-null   object 
 2   price                   1405 non-null   int64  
 3   living_complex          609 non-null    object 
 4   title                   1405 non-null   object 
 5   rooms                   1400 non-null   object 
 6   metro_time              1396 non-null   float64
 7   info_total_area         1405 non-null   float64
 8   info_living_area        1019 non-null   float64
 9   info_kitchen_area       1045 non-null   float64
 10  info_floor              1405 non-null   int64  
 11  info_total_floors       1405 non-null   int64  
 12  info_year_built         1016 non-null   float64
 13  rent_price_per_month    1405 non-null   int64  
 14  rent_jkh                1405 non-null   